In [1]:
import pandas as pd
from Stocks import Stock
from Portfolio import Portfolio
import plotter as plot
import matplotlib.pyplot as plt
import yaml
import sys
from pathlib import Path
import distributions as dt
import returns as rt
import numpy as np

import scipy.stats as stats
from prophet import Prophet
import matplotlib.pyplot as plt
import prophet_helpers as ph



def read_yaml(file_name): 
    with open(file_name) as file:
        yaml_data= yaml.safe_load(file)
    return yaml_data

config = read_yaml('config.yaml')
# config['canadian_etfs_parsed']
canadian_etfs_parsed = config['canadian_etfs_parsed']
canadian_etfs_parsed



['VFV.TO']

<img src="Dataset_info.png" alt="Screenshot" width="800" height='400'/>

In [2]:
def subtract_dates(date1, date2):
    date1_obj = datetime.strptime(date1, '%Y-%m-%d')
    date2_obj = datetime.strptime(date2, '%Y-%m-%d')
    diff = date1_obj - date2_obj
    return diff.days

from datetime import datetime, timedelta

def subtract_days(date_str, days):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    new_date_obj = date_obj - timedelta(days=days)
    return new_date_obj.strftime('%Y-%m-%d')


import pandas as pd


def subtract_trading_days(date_str, days):
    date_obj = pd.to_datetime(date_str)
    offset = pd.offsets.BDay(n=days * -1)
    new_date_obj = offset.apply(date_obj)
    return new_date_obj.strftime('%Y-%m-%d')




In [3]:
subtract_trading_days('2012-12-31', 502)

/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/2576946134.py:1: FutureWarning: BusinessDay.apply is deprecated and will be removed in a future version. Use `offset + other` instead
  subtract_trading_days('2012-12-31', 502)


'2011-01-27'

In [4]:
start_dt = '2011-01-03'
end_dt= '2015-04-13'
training_start_dt = '2011-01-03'
training_end_dt = '2012-12-31'

test_start_dt = '2013-01-02' 
test_end_dt = '2013-12-31'

out_of_sample_start_dt = '2014-01-02'
out_of_sample_end_dt = '2015-04-13'

dia_df = Stock('DIA', start_dt, end_dt)
# dia_df.ticker_data['ticker'] = 'diaA'

spy_df= Stock('SPY',start_dt, end_dt)
# spy_df.ticker_data['ticker'] = 'SPY'

qqq_df = Stock('QQQ', start_dt, end_dt)
# qqq_df.ticker_data['ticker'] = 'QQQ'

data from alpaca
Data from alpaca not available, pulling from yahoo
[*********************100%***********************]  1 of 1 completed
data from alpaca
Data from alpaca not available, pulling from yahoo
[*********************100%***********************]  1 of 1 completed
data from alpaca
Data from alpaca not available, pulling from yahoo
[*********************100%***********************]  1 of 1 completed


In [5]:
dia_df.ticker_data

,Open,High,Low,Close,Volume,ticker,pct_change,Cumulative_Percentage_Change,normalized_close,cumulative_mean_normalized_close,cumulative_variance_normalized_close,cumulative_std_normalized_close,skew,kurtosis,Jarque_Bera_stat,Jarque_Bera_p_val
2011-01-03,116.410004,116.860001,116.360001,116.410004,9108900,DIA,NaN,0.000000,0.131682,0.131682,0.000000,0.000000,-0.426358,4.152463,804.160704,2.391784e-175
2011-01-04,116.709999,116.730003,116.110001,116.639999,9775600,DIA,0.001976,0.197574,0.134696,0.133189,0.000002,0.001507,-0.426358,4.152463,804.160704,2.391784e-175
2011-01-05,116.459999,117.190002,116.300003,117.040001,7567800,DIA,0.003429,0.541188,0.139937,0.135439,0.000012,0.003411,-0.426358,4.152463,804.160704,2.391784e-175
2011-01-06,117.139999,117.190002,116.500000,116.779999,7161600,DIA,-0.002221,0.317838,0.136530,0.135711,0.000009,0.002991,-0.426358,4.152463,804.160704,2.391784e-175
2011-01-07,116.910004,117.099998,115.820000,116.570000,9249800,DIA,-0.001798,0.137442,0.133779,0.135325,0.000008,0.002785,-0.426358,4.152463,804.160704,2.391784e-175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-06,176.320007,179.169998,176.089996,178.589996,6284900,DIA,0.007105,53.414647,0.946410,0.490137,0.073307,0.270753,-0.426358,4.152463,804.160704,2.391784e-175
2015-04-07,178.770004,179.580002,178.419998,178.419998,6011600,DIA,-0.000952,53.268613,0.944182,0.490561,0.073431,0.270982,-0.426358,4.152463,804.160704,2.391784e-175
2015-04-08,178.600006,179.559998,177.960007,178.750000,4738500,DIA,0.001850,53.552096,0.948506,0.490988,0.073558,0.271216,-0.426358,4.152463,804.160704,2.391784e-175
2015-04-09,178.660004,179.630005,177.979996,179.399994,4483800,DIA,0.003636,54.110462,0.957023,0.491422,0.073692,0.271462,-0.426358,4.152463,804.160704,2.391784e-175


In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm

class Portfolio:
    def __init__(self, tickers, start_dt, end_dt, timeframe="1D", var_confidence_level=0.95, cvar_confidence_level=0.95):
        self.stocks = {}
        self.summary_statistics = pd.DataFrame(columns=['ticker', 'mean', 'variance', 'skewness', 'kurtosis', 'VaR', 'cVaR'])
        self.pct_changes = pd.DataFrame()
        self.all_stocks_df = pd.DataFrame()

        for ticker in tickers:
            stock = Stock(ticker, start_dt, end_dt, timeframe)
            self.stocks[ticker] = stock

            mean = stock.ticker_data['pct_change'].mean()
            variance = stock.ticker_data['pct_change'].var(ddof=0)
            skewness = stock.ticker_data['skew'].iloc[0]
            kurtosis = stock.ticker_data['kurtosis'].iloc[0]

            # Calculate VaR and cVaR
            var = self.calculate_var(stock.ticker_data['pct_change'].dropna(), var_confidence_level)
            cvar = self.calculate_cvar(stock.ticker_data['pct_change'].dropna(), cvar_confidence_level)

            self.summary_statistics = self.summary_statistics.append({
                'ticker': ticker,
                'mean': mean,
                'variance': variance,
                'skewness': skewness,
                'kurtosis': kurtosis,
                'VaR': var,
                'cVaR': cvar
            }, ignore_index=True)

            self.pct_changes[ticker] = stock.ticker_data['pct_change']
            self.all_stocks_df = self.all_stocks_df.append(stock.ticker_data)

        self.correlation_matrix = self.pct_changes.corr()

    def display_summary_statistics(self):
        print(self.summary_statistics)

    def display_correlation_matrix(self):
        print(self.correlation_matrix)

    def calculate_var(self, returns, confidence_level):
        # Calculate the Value at Risk (VaR) using the parametric method
        mean = returns.mean()
        std_dev = returns.std(ddof=0)
        z_score = norm.ppf(confidence_level)
        var = mean - z_score * std_dev
        return var

    def calculate_cvar(self, returns, confidence_level):
        # Calculate the Conditional Value at Risk (cVaR) using the historical method
        var = self.calculate_var(returns, confidence_level)
        losses_below_var = returns[returns < var]
        cvar = losses_below_var.mean()
        return cvar


In [12]:
p = Portfolio(tickers=[ 'DIA', 'QQQ', 'SPY' ], start_dt=start_dt, end_dt=end_dt)
p

data from alpaca
Data from alpaca not available, pulling from yahoo
[*********************100%***********************]  1 of 1 completed
data from alpaca
Data from alpaca not available, pulling from yahoo


/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.summary_statistics = self.summary_statistics.append({
/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.all_stocks_df = self.all_stocks_df.append(stock.ticker_data)


[*********************100%***********************]  1 of 1 completed
data from alpaca
Data from alpaca not available, pulling from yahoo


/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.summary_statistics = self.summary_statistics.append({
/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.all_stocks_df = self.all_stocks_df.append(stock.ticker_data)


[*********************100%***********************]  1 of 1 completed


/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.summary_statistics = self.summary_statistics.append({
/var/folders/tp/tgqd8prn2x1g474v7zwrxq7r0000gn/T/ipykernel_14411/467402769.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.all_stocks_df = self.all_stocks_df.append(stock.ticker_data)


In [13]:
p.summary_statistics

,ticker,mean,variance,skewness,kurtosis,VaR,cVaR
0,DIA,0.000448,0.000078,-0.426358,4.152463,-0.014124,-0.021020
1,QQQ,0.000677,0.000111,-0.319222,3.023044,-0.016630,-0.024192
2,SPY,0.000515,0.000092,-0.453330,4.917879,-0.015261,-0.022763


In [14]:
p.correlation_matrix

,DIA,QQQ,SPY
DIA,1.000000,0.871865,0.971892
QQQ,0.871865,1.000000,0.926446
SPY,0.971892,0.926446,1.000000


In [18]:
p.all_stocks_

In [10]:
break 

SyntaxError: 'break' outside loop (371449011.py, line 1)

In [ ]:
df = pd.concat([dia_df.ticker_data, spy_df.ticker_data, qqq_df.ticker_data])
df.head()

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'Close', title='Closing Price across ETFs', yaxis_title='Closing Price (USD)', xaxis_title='Trading Date')




In [ ]:
plot.plot_column_across_tickers(df, col_name = 'Cumulative_Percentage_Change', title='Cumulative Percentage Change of Closing Price', xaxis_title='Trading Date', yaxis_title='Cumulative % increase')

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'Close', title='Cumulative Percentage Change of Closing Price')

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'normalized_close', title='Normalized Closing Price')

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'cumulative_mean_normalized_close', title='cumulative_mean_normalized_close')

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'cumulative_variance_normalized_close', title='cumulative_variance_normalized_close')

In [ ]:
plot.plot_column_across_tickers(df, col_name = 'cumulative_std_normalized_close', title='cumulative_std_normalized_close')

In [ ]:
x_data = [list(dia_df.ticker_data.index),list(spy_df.ticker_data.index),list(qqq_df.ticker_data.index)]
y_data = [list(dia_df.ticker_data['pct_change']), list(spy_df.ticker_data['pct_change']), list(qqq_df.ticker_data['pct_change'])]

plot.plot_multi_line_chart(x_data, y_data, labels=['DIA', 'SPY', 'QQQ'])

In [ ]:




agg_stats_df = []
tickers = list(df['ticker'].unique())

for i in range(0,len(tickers)):
    temp_df = df[df['ticker']==tickers[i]]
    
    

    price_data = temp_df['Close']
    returns = (price_data / price_data.shift(1)) - 1
    returns = returns.iloc[1:]
    
    VaR = rt.historical_var(returns, confidence_level=0.95)
    cVaR = rt.historical_cvar(returns, confidence_level=0.95)
    
    print(f"Historical VaR :{VaR}")
    print(f"Historical cVaR :{cVaR}")

    agg_stats_df.append(pd.DataFrame({'ticker':[tickers[i]], 'mean':[temp_df['normalized_close'].mean()], 'std_dev':[temp_df['normalized_close'].std()], \
                                     'VaR':[VaR], 'cVaR':[cVaR]}))
    
agg_stats_df = pd.concat(agg_stats_df)
agg_stats_df



In [ ]:
plot.plot_markowitz_bullet(list(agg_stats_df['std_dev']), list(agg_stats_df['mean']), text_vals =list(agg_stats_df['ticker']),  title='Markowitz Bullet', xaxis_title='Risk', yaxis_title='Return')

In [ ]:
plot.plot_markowitz_bullet(list(agg_stats_df['VaR']), list(agg_stats_df['mean']), text_vals =list(agg_stats_df['ticker']),  title='Markowitz Bullet using VaR as measure of Risk', xaxis_title='Risk', yaxis_title='Return')


In [ ]:
plot.plot_markowitz_bullet(list(agg_stats_df['cVaR']), list(agg_stats_df['mean']), text_vals =list(agg_stats_df['ticker']),  title='Markowitz Bullet by cVaR as a measure of Risk', xaxis_title='Risk', yaxis_title='Return')

In [ ]:
low_bound, mean, high_bound = dt.get_confidence_interval(dia_df.ticker_data, 'pct_change', conf_interval=95)
low_bound, mean, high_bound 

In [ ]:
low_bound, mean, high_bound = dt.get_confidence_interval(spy_df.ticker_data, 'pct_change', conf_interval=95)
low_bound, mean, high_bound 

In [ ]:
low_bound, mean, high_bound = dt.get_confidence_interval(qqq_df.ticker_data, 'pct_change', conf_interval=95)
low_bound, mean, high_bound 

In [ ]:
dia_df = Stock('DIA', start_dt, end_dt)
# dia_df.ticker_data['ticker'] = 'diaA'

spy_df= Stock('SPY', start_dt, end_dt)
# spy_df.ticker_data['ticker'] = 'SPY'

qqq_df = Stock('QQQ', start_dt, end_dt)
# qqq_df.ticker_data['ticker'] = 'QQQ'

In [ ]:
dfs = [dia_df.ticker_data,spy_df.ticker_data, qqq_df.ticker_data]


def merge_dfs(dfs): 

    merged_df = dfs[0].copy(deep=True)
    ticker = merged_df['ticker'].iloc[0]
    merged_df.rename(columns={'Close':f'Close_{ticker}'}, inplace=True)

    for i in range(1,len(dfs)): 
        temp_df =dfs[i].copy(deep=True)
        temp_ticker = temp_df['ticker'].iloc[0]
        temp_df.rename(columns={'Close':f'Close_{temp_ticker}'}, inplace=True)


        merged_df = pd.merge(merged_df, temp_df, left_index=True, right_index=True)
        
    return merged_df 
    
merged_df =merge_dfs(dfs)
merged_df.head()

In [ ]:
# dia_df.ticker_data.head()

In [ ]:
irrelevant_cols = [col for col in merged_df.columns if any(x in col.lower() for x in ['open', 'high', 'low', 'volume'])]
irrelevant_cols

merged_df.drop(columns=irrelevant_cols, inplace=True)
close_cols = [f'Close_{ticker}' for ticker in tickers]
close_cols

In [ ]:
merged_df = merged_df[close_cols]
merged_df.head()

In [ ]:
def generate_combinations(step=0.2, target_sum=1.0, num_variables=3, precision=1):
    combinations = []
    num_steps = int(target_sum / step) + 1

    for i in range(num_steps):
        for j in range(num_steps - i):
            k = num_steps - i - j - 1
            if round(i * step, precision) + round(j * step, precision) + round(k * step, precision) == target_sum:
                combinations.append((round(i * step, precision), round(j * step, precision), round(k * step, precision)))

    return combinations

combinations = generate_combinations()
print(combinations)


In [ ]:
i =7
combination = combinations[i]
combination

### Simulation of Portfolio performance under naive weights

In [ ]:
initial_budget =10000

def simulate_portfolio_value(input_df, combination): 

    for i in range(0,len(combination)): 
        input_df[f'weight_{tickers[i]}'] = combination[i]

    for ticker in tickers: 
        input_df[f'{ticker}_num_stocks'] = initial_budget*input_df[f'weight_{ticker}'].iloc[0]/input_df[f'Close_{ticker}'].iloc[0]

    portfolio_values = []
    for i in range(0,len(input_df)): 
        portfolio_value = 0
        for ticker in tickers: 
            portfolio_value+= input_df[f'Close_{ticker}'].iloc[i] * input_df[f'{ticker}_num_stocks'].iloc[0]
        portfolio_values.append(portfolio_value)
    portfolio_values

    input_df['portfolio_value'] = portfolio_values

    return input_df

In [ ]:
plt.figure(figsize=(20,10))

simulated_portfolio_values = []
for i in range(0,len(combinations)): 
    simulated_portfolio_values.append((combinations[i], simulate_portfolio_value(merged_df, combinations[i])['portfolio_value']))
simulated_portfolio_values

for i in range(0,len(simulated_portfolio_values)): 
    plt.plot(pd.to_datetime(simulated_portfolio_values[i][1].index), simulated_portfolio_values[i][1].values, label=str(simulated_portfolio_values[i][0]))
    
    
plt.axhline(y=10000, color='r',  ls='--', label=f'simulated initial budget of ${initial_budget}')

plt.legend(loc='upper right', title='Weights for dia, SPY and QQQ resp.', bbox_to_anchor=(1.2, 1.0))
plt.xticks(rotation=90)
plt.title(f"Simulation of portfolio value under different weights for stocks dia, SPY and QQQ from {start_dt} to {end_dt}", fontsize=20)
plt.xlabel("Trading Date")
plt.ylabel("Portfolio Value in USD $")



In [ ]:
i = 0
simulated_portfolio_values_df_list = []

simulated_portfolio_values[i]

for i in range(0,len(simulated_portfolio_values)): 
    
    price_data =simulated_portfolio_values[i][1]
    returns = (price_data / price_data.shift(1)) - 1
    returns = returns.iloc[1:]
    VaR = rt.historical_var(returns, confidence_level=0.95)
    cVaR = rt.historical_cvar(returns, confidence_level=0.95)
    
    simulated_portfolio_values_df_list.append(pd.DataFrame({'portfolio_config':[str(simulated_portfolio_values[i][0])], \
                                                            'mean':[np.mean(simulated_portfolio_values[i][1])], \
                                                            'var':[np.var(simulated_portfolio_values[i][1])], \
                                                            'std':[np.std(simulated_portfolio_values[i][1])],\
                                                            'VaR':[VaR], 'cVaR':[cVaR]
                                                           }))
    
simulated_portfolio_values_df = pd.concat(simulated_portfolio_values_df_list)
simulated_portfolio_values_df['net_return'] = (simulated_portfolio_values_df['mean']-initial_budget)/initial_budget
simulated_portfolio_values_df

In [ ]:
plot.plot_markowitz_bullet(list(simulated_portfolio_values_df['var']), list(simulated_portfolio_values_df['net_return']), \
                           text_vals =list(simulated_portfolio_values_df['portfolio_config']), \
                           title='Markowitz Bullet for different portfolios under diff weights for (diaA, SPY, QQQ)',\
                           xaxis_title='Variance(Risk)', yaxis_title='Return from initial investment (%)', height=600, width=800)

In [ ]:
plot.plot_markowitz_bullet(list(simulated_portfolio_values_df['std']), list(simulated_portfolio_values_df['net_return']), \
                           text_vals =list(simulated_portfolio_values_df['portfolio_config']), \
                           title='Markowitz Bullet for different portfolios under diff weights for (diaA, SPY, QQQ)',\
                           xaxis_title='Standard Deviation (Risk)', yaxis_title='Return from initial investment (%)', height=600, width=800)

In [ ]:
plot.plot_markowitz_bullet(list(simulated_portfolio_values_df['VaR']), list(simulated_portfolio_values_df['net_return']), \
                           text_vals =list(simulated_portfolio_values_df['portfolio_config']), \
                           title='Markowitz Bullet (VaR) for different portfolios under diff weights for (diaA, SPY, QQQ)',\
                           xaxis_title='Standard Deviation (Risk)', yaxis_title='Return from initial investment (%)', height=600, width=800)

In [ ]:
plot.plot_markowitz_bullet(list(simulated_portfolio_values_df['cVaR']), list(simulated_portfolio_values_df['net_return']), \
                           text_vals =list(simulated_portfolio_values_df['portfolio_config']), \
                           title='Markowitz Bullet (cVaR) for different portfolios under diff weights for (diaA, SPY, QQQ)',\
                           xaxis_title='Standard Deviation (Risk)', yaxis_title='Return from initial investment (%)', height=600, width=800)

In [ ]:
# import pandas as pd
# import plotly.graph_objects as go

# def multiline_plot(df, columns, labels=None):
#     """
#     Creates a multiline plot using Plotly.

#     Args:
#     df (pd.DataFrame): The input DataFrame.
#     columns (list): A list of column names to plot.
#     labels (list, optional): A list of labels corresponding to the columns. Default is None, in which case column names are used.

#     Returns:
#     plotly.graph_objects.Figure: The multiline plot figure.
#     """

#     if labels is None:
#         labels = columns

#     assert len(columns) == len(labels), "The number of columns and labels must be equal."

#     fig = go.Figure()

#     for col, label in zip(columns, labels):
#         fig.add_trace(go.Scatter(x=df.index, y=df[col], name=label, mode='lines'))

#     fig.update_layout(title='Multiline Plot', xaxis_title='Date', yaxis_title='Values')

#     return fig

# Example usage
columns_to_plot = ['Close_DIA', 'Close_SPY', 'Close_QQQ', 'portfolio_value']
labels = ['dia',  'NASDAQ', 'S&P 500','portfolio value']

fig = plot.multiline_plot(merged_df, columns_to_plot, labels)
fig.show()


### Forecasting Using Facebook Prophet

In [ ]:
!pip install prophet

In [ ]:
dia_df.ticker_data

In [ ]:
ph.plot_prediction(dia_df, ticker='DIA', test_period=30)

In [ ]:
ph.plot_prediction(qqq_df, ticker='QQQ', test_period=30)

In [ ]:
ph.plot_prediction(spy_df, ticker='SPY', test_period=30)

In [ ]:

# Example usage:
data = np.random.normal(0, 1, 1000)  # Generate a sample normal distribution
dt.generate_qq_plot_normal(data, title='Theoretical Perfect Gaussian', label='Artificial Perfect Gaussian')

dt.generate_qq_plot_normal(spy_df.ticker_data['pct_change'], title='QQ plot on SPY', label='SPY returns')

In [ ]:
dt.generate_qq_plot_normal(dia_df.ticker_data['pct_change'], title='QQ plot on dia', label='dia returns')

In [ ]:
dt.generate_qq_plot_normal(qqq_df.ticker_data['pct_change'], title='QQ plot on QQQ ETF', label='QQQ returns')